In [ ]:
#! pip uninstall torch torchvision -y
#! pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
! pip install -U transformers
! pip install -U simpletransformers  

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir("/content/drive/MyDrive/Data/dataset")

In [ ]:
!pip install contractions
!pip install transformers

In [5]:
# Importing the libraries needed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import seaborn as sns
import pandas as pd
import transformers
import re
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
import logging
logging.basicConfig(level=logging.ERROR)

In [6]:
data_raw = pd.read_csv("data_467.csv")

In [7]:
data_raw.head()

,Unnamed: 0,BROWSE_NODE_ID,Text
0,5,6,"['glance', 'women', 'wallet', 'black', 'this',..."
1,55,76,"['scent', 'shield', 'men', 'deodorant', 'pill'..."
2,79,108,"['realtree', 'advantage', 'classic', 'sheet', ..."
3,92,82,"['textile', 'fabric', 'paint', 'borduex', 'lig..."
4,171,215,"['retro', 'donald', 'duck', 'ballpointretro', ..."


In [8]:
data_raw = data_raw.drop(columns=['Unnamed: 0'])

In [9]:
def processText(df):
  df = df.str.replace(r'[^a-z]', " ")
  df = df.apply(lambda x: ' '.join(x.split()))
  return df

In [10]:
data_raw['Text'] = processText(data_raw['Text'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [11]:
label = data_raw['BROWSE_NODE_ID'].values
data = data_raw['Text'].values

In [ ]:
def standardize_text(data):
  data = ' '.join(data)
  data = contractions.fix(data)
  data = re.sub(r"[^A-Za-z]", " ", data)
  data = data.lower()
  return data

In [ ]:
data_processed = []
for d in data:
  data_processed.append(standardize_text(d))

In [ ]:
data_raw.columns

Index(['TITLE', 'DESCRIPTION', 'BULLET_POINTS', 'BRAND', 'BROWSE_NODE_ID'], dtype='object')

In [12]:
from collections import Counter

In [13]:
temp = list(Counter(data_raw.BROWSE_NODE_ID.values).keys())

In [14]:
key = {}
for i in range(len(temp)):
  key[temp[i]] = i;

In [15]:
label_mod = [key[val] for val in label]

In [ ]:
print(f"BROWSE_NODE_ID column has",  data_raw.BROWSE_NODE_ID.isnull().sum()/data_raw.shape[0]*100)
print(f"TITLE column has", data_raw.TITLE.isnull().sum()/data_raw.shape[0]*100)
print(f"DESCRIPTION column has", data_raw.DESCRIPTION.isnull().sum()/data_raw.shape[0]*100)
print(f"BULLET_POINTS column has", data_raw.BULLET_POINTS.isnull().sum()/data_raw.shape[0]*100)
print(f"BRAND column has", data_raw.BRAND.isnull().sum()/data_raw.shape[0]*100)

BROWSE_NODE_ID column has 0.0
TITLE column has 0.0
DESCRIPTION column has 0.0
BULLET_POINTS column has 0.0
BRAND column has 0.0


In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, label_mod, test_size=0.2, random_state=42, stratify=label_mod)

In [17]:
train_df = pd.DataFrame(X_train)
train_df['target'] = y_train

eval_df = pd.DataFrame(X_test)
eval_df['target'] = y_test

train_df.shape, eval_df.shape

((37360, 2), (9340, 2))

In [ ]:
!pip install simpletransformers

In [20]:
from simpletransformers.classification import ClassificationModel
import logging
import sklearn

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# They are lot of arguments to play with
custom_args = {
   'output_dir': 'outputs/',
   'cache_dir': 'cache/',
   'fp16_opt_level': 'O1',
   'max_seq_length': 128,
   'train_batch_size': 16,
   'eval_batch_size': 16,
   'num_train_epochs': 1,
   'learning_rate': 1e-05,
   'logging_steps': 50,
   'save_steps': 2000,
   'overwrite_output_dir': True,
}

# Create a ClassificationModel
model = ClassificationModel('xlnet', 'xlnet-base-cased', num_labels = 467, args=custom_args) # You can set class weights by using the optional weight argument


Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'logits_proj.bias', 'sequence_summary.summary.weight', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [21]:
EPOCHS = 10
for i in range(EPOCHS):
  model.train_model(train_df)
  result, model_outputs, wrong_predictions = model.eval_model(eval_df, acc=sklearn.metrics.accuracy_score)
  print(result)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:602: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/37360 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/2335 [00:00<?, ?it/s]

...

Running Epoch 0 of 1:   0%|          | 0/2335 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1443: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/9340 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/584 [00:00<?, ?it/s]

{'mcc': 0.8115135958345101, 'acc': 0.8118843683083512, 'eval_loss': 0.7884499948114565}
